# Linear model
We'll train a linear model for images of 10 digits  
$ f(x) = x * A + b $ - **find a function that maps image x to 10 scores**  
Assuming images 28x28x1 (grayscale), $x$ has dimensionality $28*28 = 784$  
$ x \in R^{1x784} $  - image  
$ f(x) \in R^{1x10} $  - scores for each digit  
$ A \in R^{784x10} $  - linear transformation  
$ b \in R^{1x10} $ -  bias term

We want to find **A, b** that will assign a **high score** to the correct digit.  
A score for digit $i$:  
$x * A_{:,i} + b_i = \sum_{j=1}^{784}x_j*A_{j,i} + b_i$ - *weighted sum of pixels*

# PyTorch datasets
Default implementations available: https://pytorch.org/docs/stable/data.html  
For images: https://pytorch.org/docs/stable/torchvision/datasets.html#imagefolder  
**Dataset** must implement __getitem__(index) (returns indexed element) and __len__() (returns number of elements)  
Some datasets have an interface ready in `torchvision` package and are downloaded automatically

<img src="https://camo.githubusercontent.com/d440ac2eee1cb3ea33340a2c5f6f15a0878e9275/687474703a2f2f692e7974696d672e636f6d2f76692f3051493378675875422d512f687164656661756c742e6a7067">

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

**MNIST** dataset ready in https://pytorch.org/docs/stable/torchvision/datasets.html#mnist

In [ ]:
from torchvision.datasets import MNIST
# Create MNIST dataset
root_dir = '../mnist'
dataset = MNIST(root_dir, train=True, download=True)

In [ ]:
print(len(dataset))

In [ ]:
# Show elements of dataset
image, label = dataset[0]
print(image)
print(label)

In [ ]:
# Create MNIST dataset that transforms images to Tensors
from torchvision.transforms import ToTensor, Normalize, Compose
transforms = Compose([
    ToTensor(),
    Normalize(mean=(0.5,), std=(0.5,))
])
dataset = MNIST(root_dir, train=True, download=True, transform=transforms)

In [ ]:
# Show elements of dataset
image, label = dataset[0]
print(image.size())
label

**DataLoader** objects provide a way to efficiently iterate over datasets https://pytorch.org/docs/stable/data.html

In [ ]:
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4) #num_workers = n - how many threads in background for efficient loading

In [ ]:
#We can iterate over the dataset
for xs, ys in dataloader:
    print(xs.size(), ys.size())
    print(ys)
    break

## PyTorch models
Pytorch models are defined as **Module** objects that need to have a **forward** method implemented that applies transformations (neural network layers) on data. https://pytorch.org/docs/stable/nn.html

In [ ]:
import torch.nn as nn
class LinearModel(nn.Module):
    
    def __init__(self, input_dim, n_classes):
        super(LinearModel, self).__init__()
        self.fc = nn.Linear(input_dim, n_classes)
        
    def forward(self, x):
        # Apply linear transform to flattened image
        # x - batch of images Nx1x28x28
        x = x.view(x.size(0), 784)
        out = self.fc(x)
        return out

In [ ]:
# Create model
model = LinearModel(input_dim=784, n_classes=10)
model = model.to(device)

In [ ]:
from utils import train
train(dataloader, model, n_epochs=50, device=device)

In [ ]:
# torch.save(model.state_dict(), 'model_tmp.ckpt')
# model.load_state_dict(torch.load('model.ckpt'))

In [ ]:
# Create test dataset and test data loader
test_dataset = MNIST(root_dir, train=False, download=True, transform=transforms)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

In [ ]:
# Write function predict(dataloader, model) that will return predictions and labels for the entire dataset
def predict(dataloader, model):
    model.eval()
    with torch.no_grad():
        predictions = []
        labels = []
        for data, target in dataloader:
            data = data.to(device)
            output = model(data)
            prediction = output.cpu().numpy().argmax(1)
            predictions.extend(prediction)
            labels.extend(target.numpy())
    predictions = np.array(predictions)
    labels = np.array(labels)
    return predictions, labels

In [ ]:
# Evaluate accuracy on test set and train set
import numpy as np
predictions, labels = predict(test_dataloader, model)

In [ ]:
accuracy = 100 * (np.sum(predictions==labels) / len(labels))
print('Test accuracy: {:.2f}%'.format(accuracy))

In [ ]:
predictions, labels = predict(dataloader, model)
accuracy = 100 * (np.sum(predictions==labels) / len(labels))
print('Train accuracy: {:.2f}%'.format(accuracy))

In [ ]:
# Visualize learned weights
digit_weights = []
for i in range(10):
    digit_weight = model.fc.weight[i].view(28, 28).detach().cpu().numpy()
    digit_weight = digit_weight - digit_weight.min()
    digit_weight = digit_weight / digit_weight.max() # it's in [0; 1] now
    digit_weights.append(digit_weight)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
for i in range(10):
    plt.figure()
    plt.imshow(digit_weights[i], cmap='gray')

In [ ]:
# Show an image and prediction 
img = (test_dataset[0][0] * 0.5 + 0.5).numpy()[0]

In [ ]:
plt.imshow(img, cmap='gray')

In [ ]:
output = model(test_dataset[0][0].to(device))
print(output)

In [ ]:
print(output.argmax())